In [1]:
import json
from urllib.request import urlopen
import requests,json,os,sys,time,re
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
def show_work_status(singleCount, totalCount, currentCount):
    currentCount+=singleCount
    percentage=1.*currentCount/totalCount*100
    status='>'*int(percentage/2)+'-'*(50-int(percentage/2))
    if percentage % 3:
        sys.stdout.write('\rStatus:[{0}]{1:.2f}%'.format(status,percentage))
        sys.stdout.flush()
    if percentage>=100: print('\n')

* Import dataset from function1 and take out top 100 game to deal with

In [3]:
import string
content_data = pd.read_csv('part2.csv').head(100)
del content_data['Unnamed: 0']  
content_data

,Positive,Negative,Game_id,Game_name,Total_rating,Review_score,Rating
0,265404,3235,620,Portal 2,268639,0.987958,97.663280
1,163645,2138,1145360,Hades,165783,0.987104,97.403041
2,129738,1587,427520,Factorio,131325,0.987915,97.386900
3,71294,805,1118200,People Playground,72099,0.988835,97.197789
4,817296,17421,105600,Terraria,834717,0.979129,97.122464
...,...,...,...,...,...,...,...
95,23566,886,386940,Ultimate Chicken Horse,24452,0.963766,94.162057
96,87559,4027,221380,Age of Empires II (2013),91586,0.956030,94.139741
97,35548,1466,500,Left 4 Dead,37014,0.960393,94.098851
98,27705,1101,418240,Shadow Tactics: Blades of the Shogun,28806,0.961779,94.078982


* Add website links to each game by its game_id, but first, we need to convert game_id into str, then we can concate game_id with pre_link as string

In [4]:
content_data['Game_id'] = content_data['Game_id'].apply(str)  
content_data['links'] = ('https://store.steampowered.com/app/'+ content_data['Game_id']).astype(str)
content_data.head(5)

,Positive,Negative,Game_id,Game_name,Total_rating,Review_score,Rating,links
0,265404,3235,620,Portal 2,268639,0.987958,97.663280,https://store.steampowered.com/app/620
1,163645,2138,1145360,Hades,165783,0.987104,97.403041,https://store.steampowered.com/app/1145360
2,129738,1587,427520,Factorio,131325,0.987915,97.386900,https://store.steampowered.com/app/427520
3,71294,805,1118200,People Playground,72099,0.988835,97.197789,https://store.steampowered.com/app/1118200
4,817296,17421,105600,Terraria,834717,0.979129,97.122464,https://store.steampowered.com/app/105600


* In this step, we want to use http_requests and Beautiful_Soup as our tools to parse these games' descriptions on Steam website

In [5]:
import urllib
import bs4
from bs4 import BeautifulSoup

Game_description = pd.Series([]) 
current_count = 0
for i in range(100):
    url = content_data['links'][i]
    url_contents = urllib.request.urlopen(url).read()
    soup = bs4.BeautifulSoup(url_contents, 'html.parser')
    div = soup.find("div", {"id": "game_area_description"})
    
    # Find area of game description
    Game_description[i]=div
    show_work_status(1,100,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



* Get the whole content about game_descriptions with html format

In [6]:
soup = bs4.BeautifulSoup(url_contents, 'html.parser')
div = soup.find("div", {"id": "game_area_description"})

content = str(div)
content_data['Game_description'] = Game_description
content_data['Game_description'] = content_data['Game_description'].apply(str)
content_data

,Positive,Negative,Game_id,Game_name,Total_rating,Review_score,Rating,links,Game_description
0,265404,3235,620,Portal 2,268639,0.987958,97.663280,https://store.steampowered.com/app/620,"<div class=""game_area_description"" id=""game_ar..."
1,163645,2138,1145360,Hades,165783,0.987104,97.403041,https://store.steampowered.com/app/1145360,"<div class=""game_area_description"" id=""game_ar..."
2,129738,1587,427520,Factorio,131325,0.987915,97.386900,https://store.steampowered.com/app/427520,"<div class=""game_area_description"" id=""game_ar..."
3,71294,805,1118200,People Playground,72099,0.988835,97.197789,https://store.steampowered.com/app/1118200,"<div class=""game_area_description"" id=""game_ar..."
4,817296,17421,105600,Terraria,834717,0.979129,97.122464,https://store.steampowered.com/app/105600,"<div class=""game_area_description"" id=""game_ar..."
...,...,...,...,...,...,...,...,...,...
95,23566,886,386940,Ultimate Chicken Horse,24452,0.963766,94.162057,https://store.steampowered.com/app/386940,"<div class=""game_area_description"" id=""game_ar..."
96,87559,4027,221380,Age of Empires II (2013),91586,0.956030,94.139741,https://store.steampowered.com/app/221380,"<div class=""game_area_description"" id=""game_ar..."
97,35548,1466,500,Left 4 Dead,37014,0.960393,94.098851,https://store.steampowered.com/app/500,"<div class=""game_area_description"" id=""game_ar..."
98,27705,1101,418240,Shadow Tactics: Blades of the Shogun,28806,0.961779,94.078982,https://store.steampowered.com/app/418240,"<div class=""game_area_description"" id=""game_ar..."


* Clean the HTML content with regular expression and get the text descriptions' content

In [7]:
current_count = 0
for i in range(0,100):
    content_data['Game_description'][i] = re.sub('<.*?>', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = re.sub('\\n.*\\n', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = content_data['Game_description'][i].replace("\t","")
    content_data['Game_description'][i] = content_data['Game_description'][i].translate(content_data['Game_description'][i].maketrans(' ',' ',string.punctuation))

    show_work_status(1,100,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [8]:
content_data.head(5)

,Positive,Negative,Game_id,Game_name,Total_rating,Review_score,Rating,links,Game_description
0,265404,3235,620,Portal 2,268639,0.987958,97.663280,https://store.steampowered.com/app/620,Portal 2 draws from the awardwinning formula...
1,163645,2138,1145360,Hades,165783,0.987104,97.403041,https://store.steampowered.com/app/1145360,Hades is a godlike roguelike dungeon c...
2,129738,1587,427520,Factorio,131325,0.987915,97.386900,https://store.steampowered.com/app/427520,Factorio is a game in which you build and ...
3,71294,805,1118200,People Playground,72099,0.988835,97.197789,https://store.steampowered.com/app/1118200,Shoot stab burn poison tear vaporise or c...
4,817296,17421,105600,Terraria,834717,0.979129,97.122464,https://store.steampowered.com/app/105600,Dig Fight Explore Build The very world is a...


* Using Tf-idf to measure the weight of every word in each game description

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(list(content_data['Game_description']))

# Convert every Series in DataFrame to List format
tfidf_vectorizer.get_feature_names()

# Array mapping from feature integer indices to feature name
tfidf

<100x5099 sparse matrix of type '<class 'numpy.float64'>'
	with 11324 stored elements in Compressed Sparse Row format>

* This is description of the top 100 games

In [10]:
list_app_name = list(content_data['Game_name'])
content_data['Game_description']

0       Portal 2 draws from the awardwinning formula...
1          Hades  is a  godlike  roguelike dungeon c...
2        Factorio  is a game in which you build and ...
3          Shoot stab burn poison tear vaporise or c...
4       Dig Fight Explore Build  The very world is a...
                            ...                        
95      Ever wish you were a walljumping arrowdodgin...
96      In Age of Empires II HD Edition fans of the ...
97      From Valve the creators of CounterStrike Hal...
98         Shadow Tactics is a hardcore tactical ste...
99      The last days of man are at hand Two years a...
Name: Game_description, Length: 100, dtype: object

* Using Cosine as Similarity measures,sorting Top 10 moast relative games

In [13]:
dic_recommended_1 = {}
for index in range(content_data['Game_description'].shape[0]):
    #Iterate items in DataFrame
    print(tfidf[2])

    similarities_1 = linear_kernel(tfidf[index],tfidf).flatten()
    related_docs_indices_1 = (-similarities_1).argsort()[:10]

    dic_recommended_1.update({list_app_name[index]:[list_app_name[i] for i in related_docs_indices_1]})
    
df_content_based_results_1 = pd.DataFrame(dic_recommended_1)
df_content_based_results_1.reset_index(inplace=True)

  (0, 5083)	0.07656755686049
  (0, 5092)	0.08344133437878624
  (0, 202)	0.08344133437878624
  (0, 4058)	0.06790762134891047
  (0, 364)	0.07656755686049
  (0, 1407)	0.08344133437878624
  (0, 4378)	0.06481675657872478
  (0, 2912)	0.08344133437878624
  (0, 3055)	0.08344133437878624
  (0, 2579)	0.08344133437878624
  (0, 469)	0.08344133437878624
  (0, 1661)	0.08344133437878624
  (0, 823)	0.06481675657872478
  (0, 974)	0.14338106819404206
  (0, 3658)	0.08344133437878624
  (0, 3260)	0.07656755686049
  (0, 4095)	0.06481675657872478
  (0, 2741)	0.08344133437878624
  (0, 2680)	0.08344133437878624
  (0, 2103)	0.08344133437878624
  (0, 1943)	0.059939733815255816
  (0, 2210)	0.062203467651422005
  (0, 3918)	0.07656755686049
  (0, 3256)	0.05170900657796392
  (0, 1921)	0.059939733815255816
  :	:
  (0, 596)	0.06481675657872478
  (0, 4484)	0.06481675657872478
  (0, 3737)	0.08344133437878624
  (0, 3745)	0.09856608709769803
  (0, 2878)	0.14338106819404206
  (0, 1672)	0.12963351315744956
  (0, 2731)	0.071

* Results: the most relative 10 games about game_description contents for each top 100 popular games

In [20]:
df_content_based_results_1 = df_content_based_results_1.T
df_content_based_results_1.shape
df_content_based_results_1

,0,1,2,3,4,5,6,7,8,9
index,0,1,2,3,4,5,6,7,8,9
Portal 2,Portal 2,Portal,Half-Life,Left 4 Dead 2,Supreme Commander: Forged Alliance,Mark of the Ninja,Resident Evil 2,Left 4 Dead,Castle Crashers,Hollow Knight
Hades,Hades,HuniePop,Hollow Knight,Risk of Rain 2,Mirror,Dead Cells,Unheard,Portal 2,Stardew Valley,Divinity: Original Sin 2 - Definitive Edition
Factorio,Factorio,Dyson Sphere Program,Satisfactory,BeamNG.drive,Tabletop Simulator,Don't Starve Together,Sid Meier's Civilization V,Portal 2,American Truck Simulator,Bloons TD 6
People Playground,People Playground,RimWorld,Dyson Sphere Program,Tabletop Simulator,Garry's Mod,Portal,One Finger Death Punch,Sid Meier's Civilization V,FTL: Faster Than Light,Left 4 Dead 2
...,...,...,...,...,...,...,...,...,...,...
Ultimate Chicken Horse,Ultimate Chicken Horse,BattleBlock Theater,Castle Crashers,Duck Game,Divinity: Original Sin 2 - Definitive Edition,Tabletop Simulator,Broforce,Don't Starve Together,Keep Talking and Nobody Explodes,Beat Saber
Age of Empires II (2013),Age of Empires II (2013),Duck Game,Persona 4 Golden,Sid Meier's Civilization V,Portal 2,Stronghold Crusader HD,Ori and the Blind Forest: Definitive Edition,Hollow Knight,Resident Evil 2,Don't Starve Together
Left 4 Dead,Left 4 Dead,Left 4 Dead 2,Resident Evil 2,Counter-Strike: Source,The Walking Dead,Risk of Rain 2,Supreme Commander: Forged Alliance,Portal 2,Outlast,Half-Life: Alyx
Shadow Tactics: Blades of the Shogun,Shadow Tactics: Blades of the Shogun,Divinity: Original Sin 2 - Definitive Edition,Hollow Knight,Deep Rock Galactic,Mark of the Ninja,Celeste,Dishonored,ULTRAKILL,Slay the Spire,The Expendabros
